In [256]:
import requests
import pandas as pd
import folium

In [257]:
BASE_REQUEST = 'http://router.project-osrm.org/route/v1/foot/'

In [270]:
def getRoute(*args,points_to_include=[],points_to_avoid=[]):
    k = ';'.join(args)
    req = BASE_REQUEST + k + '?overview=full&steps=true&alternatives=3'
    _r = requests.get(req)
    _ = []
    for _route in range(len(_r.json()['routes'])):
        _tmp = pd.DataFrame()
        for _leg in range(len(_r.json()['routes'][_route]['legs'])):
            _tmp = _tmp.append(pd.json_normalize(_r.json()['routes'][_route]['legs'][_leg]['steps']))
        _.append(_tmp.reset_index(drop=True))
    
    return _
    

def getLocations(_df):
    loc = []
    for row,col in _df.iterrows():
        _ = pd.json_normalize(col['intersections'])
        for _row,_col in _.iterrows():
            loc.append((_col['location'][1],_col['location'][0]))
    
    return loc

In [299]:

coord1 = '-0.076303,51.50815' # Tower of London
coord2 = '-0.108711,51.50457' # London Waterloo East Tube Station
coord3 = '-0.124613,51.50106' # Big Ben
coord4 = '-0.60604413,51.48402' # Windsor castle

res = getRoute(coord1,coord4)

print(f'Amount of routes found: {len(res)}')
for i in range(len(res)):
    print(f'Length of route 1: {sum(res[i]["distance"])} meters')

Amount of routes found: 2
Length of route 1: 39587.2 meters
Length of route 1: 41887.6 meters


In [295]:
map = folium.Map(location=[51.5, -0.1], zoom_start=13)
colors=['#000000','#FF0000','#00FF00','#0000FF']
for routes in range(len(res)):
    l = getLocations(res[routes])
    for point in l:
        folium.CircleMarker(point, radius=5, color=colors[routes]).add_to(map)
map

In [89]:
df2 = pd.json_normalize(df.iloc[1]['intersections'])
df2

,classes,out,in,entry,location,bearings
0,[restricted],2,0,"[False, True, True]","[-0.07546, 51.507002]","[15, 105, 285]"
1,[restricted],2,1,"[False, False, True]","[-0.078667, 51.508022]","[105, 195, 345]"
2,NaN,1,0,"[False, True]","[-0.078812, 51.508206]","[105, 315]"


In [86]:
sum(df['distance'])

3586.2000000000003